In [1]:
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization, Flatten
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model, Sequential
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

In [2]:
df_labels = pd.read_csv('../content/drive/MyDrive/DSE_I2100/data/GroundTruth.csv')
df_labels['image'] = df_labels['image'] +'.jpg'

labels=['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']
label_list=[]
for i in range (len(df_labels)):
    row= list(df_labels.iloc[i])
    del row[0]
    index=np.argmax(row)
    label=labels[index]
    label_list.append(label)
df_labels['label']= label_list
df_labels=df_labels.drop(labels, axis=1)
df_labels['label'].value_counts()

NV       6705
MEL      1113
BKL      1099
BCC       514
AKIEC     327
VASC      142
DF        115
Name: label, dtype: int64

In [3]:
from sklearn.model_selection import train_test_split

tain_size=0.7
valid_size = 0.1
test_size = 0.2
valid_test_split = valid_size / test_size

train_df, valid_test_df = train_test_split(df_labels, train_size=tain_size, shuffle=True, random_state=42)
valid_df, test_df = train_test_split(valid_test_df, train_size=valid_test_split, shuffle=True, random_state=42)

print(' train_df length: ', len(train_df), '  test_df length: ', len(test_df), '  valid_df length: ', len(valid_df))
print (train_df.head())
print (train_df['label'].value_counts())

 train_df length:  7010   test_df length:  1503   valid_df length:  1502
                 image label
2399  ISIC_0026705.jpg    NV
8246  ISIC_0032552.jpg   MEL
8052  ISIC_0032358.jpg   BKL
5840  ISIC_0030146.jpg    NV
3540  ISIC_0027846.jpg   BCC
NV       4686
BKL       784
MEL       758
BCC       362
AKIEC     235
VASC      100
DF         85
Name: label, dtype: int64


In [35]:
sdir=r'../content/drive/MyDrive/DSE_I2100/data/images' # main directory where data is stored
save_dir=r'./' # output directory where model will be saved
subject='cancer' # part of the name of the saved model
height=64  # image height
width=64   # image width
channels=3  # number of coloor channels
batch_size=40  # model batch size for training and evaluation
img_shape=(height, width, channels)
img_size=(height, width)

gen=ImageDataGenerator(rescale=1/255) # scale pixel between 0 and 1

train_gen=gen.flow_from_dataframe( train_df, sdir, x_col='image', y_col='label', target_size=img_size, class_mode='categorical',
                                    color_mode='rgb', shuffle=True, batch_size=batch_size)

Found 7010 validated image filenames belonging to 7 classes.


In [36]:
valid_gen=gen.flow_from_dataframe(valid_df, sdir, x_col='image', y_col='label', target_size=img_size, class_mode='categorical',
                                    color_mode='rgb', shuffle=True, batch_size=batch_size)

Found 1502 validated image filenames belonging to 7 classes.


In [38]:
linear_model = tf.keras.Sequential([    
   keras.layers.Flatten(input_shape=(height, width, channels)),                                                                 
   keras.layers.Dense(7, activation = 'ReLU')                                 
])
linear_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
linear_model.fit(train_gen, epochs=8, steps_per_epoch= 7010 // batch_size,  validation_data=valid_gen, validation_steps= 1502 // batch_size)

Epoch 1/8
175/175 [==============================] - 109s 620ms/step - loss: 1.4627 - accuracy: 0.6653 - val_loss: 1.4626 - val_accuracy: 0.6662
Epoch 2/8
175/175 [==============================] - 112s 639ms/step - loss: 1.4517 - accuracy: 0.6687 - val_loss: 1.4538 - val_accuracy: 0.6682
Epoch 3/8
175/175 [==============================] - 107s 611ms/step - loss: 1.4542 - accuracy: 0.6681 - val_loss: 1.4597 - val_accuracy: 0.6669
Epoch 4/8
175/175 [==============================] - 111s 635ms/step - loss: 1.4529 - accuracy: 0.6684 - val_loss: 1.4597 - val_accuracy: 0.6669
Epoch 5/8
175/175 [==============================] - 108s 616ms/step - loss: 1.4548 - accuracy: 0.6680 - val_loss: 1.4656 - val_accuracy: 0.6655
Epoch 6/8
175/175 [==============================] - 106s 608ms/step - loss: 1.4536 - accuracy: 0.6683 - val_loss: 1.4626 - val_accuracy: 0.6662
Epoch 7/8
175/175 [==============================] - 107s 613ms/step - loss: 1.4536 - accuracy: 0.6683 - val_loss: 1.4567 - val_ac

In [37]:
linear_model = tf.keras.Sequential([    
   keras.layers.Flatten(input_shape=(height, width, channels)),                                                                 
   keras.layers.Dense(7, activation = 'sigmoid')                                 
])
linear_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
linear_model.fit(train_gen, epochs=8, steps_per_epoch= 7010 // batch_size,  validation_data=valid_gen, validation_steps= 1502 // batch_size)

Epoch 1/8
175/175 [==============================] - 386s 2s/step - loss: 0.3229 - accuracy: 0.6462 - val_loss: 0.2435 - val_accuracy: 0.6453
Epoch 2/8
175/175 [==============================] - 107s 613ms/step - loss: 0.2550 - accuracy: 0.6516 - val_loss: 0.2535 - val_accuracy: 0.6169
Epoch 3/8
175/175 [==============================] - 114s 650ms/step - loss: 0.2694 - accuracy: 0.6512 - val_loss: 0.2424 - val_accuracy: 0.6378
Epoch 4/8
175/175 [==============================] - 107s 614ms/step - loss: 0.2502 - accuracy: 0.6558 - val_loss: 0.3746 - val_accuracy: 0.6655
Epoch 5/8
175/175 [==============================] - 106s 607ms/step - loss: 0.2537 - accuracy: 0.6610 - val_loss: 0.2719 - val_accuracy: 0.6426
Epoch 6/8
175/175 [==============================] - 107s 610ms/step - loss: 0.2446 - accuracy: 0.6666 - val_loss: 0.2265 - val_accuracy: 0.6703
Epoch 7/8
175/175 [==============================] - 106s 606ms/step - loss: 0.2486 - accuracy: 0.6657 - val_loss: 0.2418 - val_accur